In [1]:
import os
import cv2
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
def compute_hog(image, cell_size=8, block_size=2, bins=9):
    # Step 1: Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)

    # Step 2: Compute gradients
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=1)
    magnitude = np.sqrt(gx**2 + gy**2)
    angle = (np.arctan2(gy, gx) * 180 / np.pi) % 180

    # Step 3: Calculate histograms for cells
    h, w = gray.shape
    n_cells_x = w // cell_size
    n_cells_y = h // cell_size
    hist = np.zeros((n_cells_y, n_cells_x, bins))

    bin_width = 180 // bins

    for i in range(n_cells_y):
        for j in range(n_cells_x):
            cell_mag = magnitude[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            cell_angle = angle[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]

            for u in range(cell_size):
                for v in range(cell_size):
                    ang = cell_angle[u, v]
                    mag = cell_mag[u, v]
                    bin_idx = int(ang // bin_width) % bins
                    hist[i, j, bin_idx] += mag

    # Step 4: Normalize blocks
    normalized_blocks = []
    for i in range(n_cells_y - block_size + 1):
        for j in range(n_cells_x - block_size + 1):
            block = hist[i:i+block_size, j:j+block_size, :].flatten()
            norm = np.linalg.norm(block) + 1e-6
            normalized_block = block / norm
            normalized_blocks.append(normalized_block)

    # Step 5: Final feature vector
    hog_vector = np.concatenate(normalized_blocks)
    return hog_vector, hist

def load_images_from_folder(folder, no_images):
    images = []
    cnt = 0
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 128))
            images.append(img)
            cnt+=1
            if (cnt >= no_images):
                break
    return images

In [30]:
pos_dir = "train_64x128_H96/pos"
neg_dir = "train_64x128_H96/neg"

In [31]:
pos_images = load_images_from_folder(pos_dir,3000)
neg_images = load_images_from_folder(neg_dir,3000)

In [32]:
X = []
y = []

for img in pos_images:
    features, _ = compute_hog(img)
    X.append(features)
    y.append(1)

for img in neg_images:
    features, _ = compute_hog(img)
    X.append(features)
    y.append(0)

In [33]:
X = np.array(X)
y = np.array(y)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [35]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [36]:
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%')

Accuracy: 97.25%
